In [1]:
pwd

'/home/jupyter/final_project/splade-colBERT/splade'

Just testing inference

In [2]:
# !pip install wandb

In [3]:
# !pip install omegaconf

In [4]:
# pip install transformers

In [21]:
import os 
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [22]:
import wandb

In [6]:
import wandb
wandb.login()

wandb: Currently logged in as: jammond. Use `wandb login --relogin` to force relogin


True

In [7]:
import torch
from transformers import AutoModelForMaskedLM, AutoTokenizer
from splade.models.transformer_rep import Splade

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


Moving 0 files to the new cache system


0it [00:00, ?it/s]

In [8]:
# set the dir for trained weights

##### v2
# model_type_or_dir = "weights/splade_max"
# model_type_or_dir = "weights/distilsplade_max"

### v2bis, directly download from Hugging Face
# model_type_or_dir = "naver/splade-cocondenser-selfdistil"
model_type_or_dir = "naver/splade-cocondenser-ensembledistil"

In [9]:
# loading model and tokenizer

model = Splade(model_type_or_dir, agg="max")
model.eval()
tokenizer = AutoTokenizer.from_pretrained(model_type_or_dir)
reverse_voc = {v: k for k, v in tokenizer.vocab.items()}

In [10]:
# example document from MS MARCO passage collection (doc_id = 8003157)

doc = "Glass and Thermal Stress. Thermal Stress is created when one area of a glass pane gets hotter than an adjacent area. If the stress is too great then the glass will crack. The stress level at which the glass will break is governed by several factors."

In [11]:
# now compute the document representation
with torch.no_grad():
    doc_rep = model(d_kwargs=tokenizer(doc, return_tensors="pt"))["d_rep"].squeeze()  # (sparse) doc rep in voc space, shape (30522,)

# get the number of non-zero dimensions in the rep:
col = torch.nonzero(doc_rep).squeeze().cpu().tolist()
print("number of actual dimensions: ", len(col))

# now let's inspect the bow representation:
weights = doc_rep[col].cpu().tolist()
d = {k: v for k, v in zip(col, weights)}
sorted_d = {k: v for k, v in sorted(d.items(), key=lambda item: item[1], reverse=True)}
bow_rep = []
for k, v in sorted_d.items():
    bow_rep.append((reverse_voc[k], round(v, 2)))
print("SPLADE BOW rep:\n", bow_rep)

number of actual dimensions:  126
SPLADE BOW rep:
 [('stress', 2.25), ('glass', 2.23), ('thermal', 2.18), ('glasses', 1.65), ('pan', 1.62), ('heat', 1.56), ('stressed', 1.42), ('crack', 1.31), ('break', 1.12), ('cracked', 1.1), ('hot', 0.93), ('created', 0.9), ('factors', 0.81), ('broken', 0.73), ('caused', 0.71), ('too', 0.71), ('damage', 0.69), ('if', 0.68), ('hotter', 0.65), ('governed', 0.61), ('heating', 0.59), ('temperature', 0.59), ('adjacent', 0.59), ('cause', 0.58), ('effect', 0.57), ('fracture', 0.56), ('bradford', 0.55), ('strain', 0.53), ('hammer', 0.51), ('brian', 0.48), ('error', 0.47), ('windows', 0.45), ('will', 0.45), ('reaction', 0.42), ('create', 0.42), ('windshield', 0.41), ('heated', 0.41), ('factor', 0.4), ('cracking', 0.39), ('failure', 0.38), ('mechanical', 0.38), ('when', 0.38), ('formed', 0.38), ('bolt', 0.38), ('mechanism', 0.37), ('warm', 0.37), ('areas', 0.36), ('area', 0.36), ('energy', 0.34), ('disorder', 0.33), ('barry', 0.33), ('shock', 0.32), ('determi

/opt/conda/lib/python3.7/site-packages/torch/autocast_mode.py:162: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


Trying to run splade

In [23]:
ls

LICENSE               data/                   inference_splade.ipynb  weights/
README.md             data.tar.gz             setup.py
conda_splade_env.yml  efficient_splade_pisa/  splade/
conf/                 images/                 splade_attempt_2.ipynb


In [24]:
ls ..\

ls: cannot access '..\': No such file or directory


In [25]:
marco_path = '..\splade\data\msmarco'

In [30]:
!cd "/home/jupyter/final_project/splade-colBERT/splade"

In [31]:
!pwd

/home/jupyter/final_project/splade-colBERT/splade


In [ ]:
dataset = {'D_COLLECTION_PATH': 'data/toy_data/val_collection', 'Q_COLLECTION_PATH': 'data/toy_data/val_queries', 'QREL_PATH': 'data/toy_data/qrel/qrel.json', 'TOP_K': 20}, 'COLLECTION_PATH': 'data/toy_data/full_collection', 'Q_COLLECTION_PATH': ['data/toy_data/dev_queries'], 'EVAL_QREL_PATH': ['data/toy_data/qrel/qrel.json'], 'flops_queries': 'data/toy_data/dev_queries'}

!pwd

In [33]:
from splade import all

config = {'config': {'lr': 2e-05, 'seed': 123, 'gradient_accumulation_steps': 1, 'weight_decay': 0.01, 'validation_metrics': ['MRR@10', 'recall@100', 'recall@200', 'recall@500'], 'pretrained_no_yamlconfig': False, 'nb_iterations': 10, 'train_batch_size': 6, 'eval_batch_size': 8, 'index_retrieve_batch_size': 6, 'record_frequency': 3, 'train_monitoring_freq':
    2, 'warmup_steps': 5, 'max_length': 10, 'fp16': False, 'augment_pairs': 'in_batch_negatives', 'matching_type': 'splade', 'monitoring_ckpt': 'loss', 'loss': 'InBatchPairwiseNLL', 'regularizer': {'FLOPS': {'lambda_q': 0.0005, 'lambda_d': 0.0003, 'T': 3, 'targeted_rep': 'rep', 'reg': 'FLOPS'}}, 'tokenizer_type': 'distilbert-base-uncased', 'top_k': 5, 'threshold': 0.4, 'eval_metric': [['mrr_10', 'recall']], 'checkpoint_dir': 'data/msmarco', 'index_dir': '???', 'out_dir': '???'}, 'data': {'type': 'triplets', 'TRAIN_DATA_DIR': 'data/toy_data/triplets', 'VALIDATION_SIZE_FOR_LOSS': 20, 'VALIDATION_FULL_RANKING': {'D_COLLECTION_PATH': 'data/toy_data/val_collection', 'Q_COLLECTION_PATH': 'data/toy_data/val_queries', 'QREL_PATH': 'data/toy_data/qrel/qrel.json', 'TOP_K': 20}, 'COLLECTION_PATH': 'data/toy_data/full_collection', 'Q_COLLECTION_PATH': ['data/toy_data/dev_queries'], 'EVAL_QREL_PATH': ['data/toy_data/qrel/qrel.json'], 'flops_queries': 'data/toy_data/dev_queries'}, 'init_dict': {'model_type_or_dir': 'distilbert-base-uncased', 'model_type_or_dir_q': None, 'freeze_d_model': 0, 'agg': 'max', 'fp16': False}}


all.train_index_retrieve(config)

ModuleNotFoundError: No module named 'hydra'

In [19]:
pwd

'C:\\Users\\jammo\\DataspellProjects\\capstone\\new_capstone_dir\\splade-colBERT\\splade'

In [53]:
from splade.utils import utils

config = "..\conf\config_splade++_cocondenser_ensembledistil_monogpu.yaml"

utils.get_initialize_config(config)

ValueError: Invalid input. Supports one of [dict,list,DictConfig,ListConfig,dataclass,dataclass instance,attr class,attr class instance]

In [38]:
# with Run().context(RunConfig(nranks=1, experiment="msmarco")):
#     trainer = Trainer(
#         triples=  r"..\data\msmarco\splade_triplets",
#         queries=  r"..\data\msmarco\train_queries\queries\raw.tsv",
#         collection= r"..\data\msmarco\full_collection\raw.tsv",
#         config=config,
#     )

NameError: name 'Run' is not defined

In [ ]:
# model =
# iterations = 1
# loss =
# optimizer =
# config = "..\conf\config_splade++_cocondenser_ensembledistil_monogpu.yaml"
# scheduler =
# train_loader =
# val_loss_loader =
# val_evaluator +
# regularizer =
#


In [23]:
# trainer = SiameseTransformerTrainer(model=model, iterations=iterations, loss=loss, optimizer=optimizer,
#                                     config=config, scheduler=scheduler,
#                                     train_loader=train_loader, validation_loss_loader=val_loss_loader,
#                                     validation_evaluator=val_evaluator,
#                                     regularizer=regularizer)
# trainer.train()

 Volume in drive C is Windows
 Volume Serial Number is C85C-4698

 Directory of C:\Users\jammo\DataspellProjects\capstone\new_capstone_dir\splade-colBERT\splade

10/02/2022  01:35 PM    <DIR>          .
10/02/2022  01:35 PM    <DIR>          ..
10/01/2022  11:52 AM    <DIR>          .ipynb_checkpoints
09/26/2022  05:45 PM             6,799 conda_splade_env.yml
10/02/2022  12:10 PM    <DIR>          conf
10/01/2022  12:23 PM    <DIR>          data
09/26/2022  05:45 PM    <DIR>          efficient_splade_pisa
09/26/2022  05:45 PM    <DIR>          images
10/02/2022  12:33 PM             9,352 inference_splade.ipynb
09/26/2022  05:45 PM            33,764 LICENSE
09/26/2022  05:45 PM            17,307 README.md
09/26/2022  05:45 PM               743 setup.py
10/01/2022  10:34 AM    <DIR>          splade
10/02/2022  01:35 PM            17,556 splade_attempt_2.ipynb
09/26/2022  05:45 PM    <DIR>          weights
               6 File(s)         85,521 bytes
               9 Dir(s)  56,088,412

In [37]:
ls data\msmarco\full_collection\raw.tsv

 Volume in drive C is Windows
 Volume Serial Number is C85C-4698

 Directory of C:\Users\jammo\DataspellProjects\capstone\new_capstone_dir\splade-colBERT\splade\data\msmarco\full_collection

10/01/2022  12:23 PM    <DIR>          .
10/01/2022  12:23 PM    <DIR>          ..
04/20/2022  05:52 AM                 0 .skeleton
04/20/2022  05:56 AM     3,047,524,442 raw.tsv
               2 File(s)  3,047,524,442 bytes
               2 Dir(s)  56,086,917,120 bytes free
